<a href="https://colab.research.google.com/github/VindhyaSRajan/Master-s-Thesis/blob/master/LDA_AmazonSummarytData_LawnAndGarden_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip3 install pandas nltk gensim pyLdavis

## All the Imports go here

In [0]:
import re # We clean text using regex
import csv # To read the csv
from collections import defaultdict # For accumlating values
from nltk.corpus import stopwords # To remove stopwords
from gensim import corpora # To create corpus and dictionary for the LDA model
from gensim.models import LdaModel # To use the LDA model
import pyLDAvis.gensim # To visualise LDA model effectively
import pandas as pd
from gensim.models import CoherenceModel #Put in use to calculate the Coherency score for LDA

In [44]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##Reading the Amazon Product Review file 

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
from google.colab import drive
drive.mount('drive')

In [0]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')



In [61]:
df = pd.read_csv('/content/drive/My Drive/reviews_samples_amazonSummary.csv')

df.head()
df.shape

(993490, 10)

In [62]:
df.head()

,Unnamed: 0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,0,A2VNYWOPJ13AFP,0981850006,"Amazon Customer ""carringt0n""","[6, 7]",This was a gift for my other husband. He's ma...,5.0,Delish,1259798400,"12 3, 2009"
1,1,A20DWVV8HML3AW,0981850006,Cyndy,"[0, 0]","My husband rarely asks for anything specific, ...",5.0,Just what my husband wanted,1371081600,"06 13, 2013"
2,2,A3RVP3YBYYOPRH,0981850006,John Warren,"[9, 11]",This guy knows his stuff. I love the seriously...,5.0,Love This Show!,1257984000,"11 12, 2009"
3,3,A28XY55TP3Q90O,0981850006,"JSG ""Lover of Learning""","[1, 1]","THIS CAN BE FOUND AT: [...] FOR $19.99, also a...",5.0,"BUY ELSEWHERE, TOO EXPENSIVE",1314144000,"08 24, 2011"
4,4,A3VZW1BGUQO0V3,0981850006,"Marc G. Belisle ""Allswell2001""","[2, 2]",Quite good particularly for inspiration (DVD g...,5.0,Quite good but I *much* prefer the cookbooks,1308268800,"06 17, 2011"


In [0]:
df.to_csv('reviews_samples_amazonSummary.csv')
!cp reviews_samples_amazonSummary.csv drive/My\ Drive/

In [0]:
from itertools import islice
fileContents = defaultdict(list)
with open('/content/drive/My Drive/reviews_samples_amazonSummary.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in islice(reader, 50000): # read a row as {column1: value1, column2: value2,...}
        for (k,v) in row.items(): # go over each column name and value 
            fileContents[k].append(v) # append the value into the appropriate list

In [46]:
reviews = fileContents['summary']
type(reviews)

list

In [0]:
reviews = [re.sub(r'[^\w\s]','',str(item)) for item in reviews]

In [0]:
stopwords = set(stopwords.words('english'))
#stopwords.extend(['ok','doesnt','try','go','get','make','dont','2','first','second','place',
                  #'would','really','like','great','came','got'])

In [0]:
texts = [[word for word in document.lower().split() if word not in stopwords] for document in reviews]

##LDA Implementation

In [0]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [0]:
dictionary = corpora.Dictionary(texts)

In [52]:
print(dictionary)

Dictionary(5335 unique tokens: ['husband', 'wanted', 'love', 'show', 'buy']...)


In [0]:
#df.dtypes
summary= df['summary'].astype(str).copy() #Need the column as a string
summary= summary.apply(lambda x: x.split())

In [0]:
# So what does the dictionary look like : Dictionary(5335 unique tokens: ['husband', 'wanted', 'love', 'show', 'buy']...)

In [0]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [0]:
#DO NOT RUN THIS ONE. It is for Version 1
#NUM_TOPICS = 10 # This is an assumption. Needs to be tuned. 
#ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=20)#This might take some time.

In [0]:
#trying to log the model fitting and saving it
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import warnings
import gensim
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=corpus,
                           num_topics=20,
                           id2word=dictionary,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train4.save('lda_train4.model')

In [77]:
coherence_model_lda = CoherenceModel(model=lda_train4, texts=summary, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
coherence_lda

0.4795560192183686

In [82]:
topics = lda_train4.show_topics()
for topic in topics:
    print(topic)

(8, '0.232*"works" + 0.117*"well" + 0.072*"job" + 0.051*"worked" + 0.027*"value" + 0.019*"working" + 0.017*"okay" + 0.017*"gets" + 0.015*"done" + 0.015*"make"')
(12, '0.085*"made" + 0.065*"bad" + 0.049*"happy" + 0.047*"purchase" + 0.047*"gift" + 0.041*"thing" + 0.039*"amazing" + 0.033*"well" + 0.033*"useful" + 0.032*"squirrels"')
(2, '0.099*"better" + 0.051*"expected" + 0.039*"much" + 0.036*"last" + 0.031*"water" + 0.029*"new" + 0.029*"disappointed" + 0.027*"exactly" + 0.027*"patio" + 0.025*"solid"')
(19, '0.140*"nice" + 0.094*"work" + 0.057*"trap" + 0.035*"didnt" + 0.028*"right" + 0.026*"trimmer" + 0.024*"mole" + 0.024*"could" + 0.019*"fly" + 0.017*"problems"')
(1, '0.092*"small" + 0.087*"awesome" + 0.046*"cheap" + 0.044*"gas" + 0.038*"fantastic" + 0.034*"thrower" + 0.033*"must" + 0.031*"timer" + 0.029*"yard" + 0.026*"idea"')
(4, '0.046*"set" + 0.032*"squirrel" + 0.032*"needed" + 0.032*"back" + 0.031*"needs" + 0.029*"hummingbird" + 0.029*"super" + 0.026*"cant" + 0.023*"enough" + 0.022

In [83]:
word_dict = {};
for i in range(NUM_TOPICS):
    words = lda_train4.show_topic(i, topn = 20)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,easy,small,better,perfect,set,money,best,great,works,good
1,pool,awesome,expected,little,squirrel,worth,like,works,well,product
2,use,cheap,much,cover,needed,far,ever,product,job,really
3,quality,gas,last,snow,back,doesnt,axe,value,worked,time
4,poor,fantastic,water,replacement,needs,used,junk,tool,value,quality
5,years,thrower,new,item,hummingbird,waste,still,beware,working,pretty
6,2,must,disappointed,plastic,super,advertised,birds,im,okay,wonderful
7,long,timer,exactly,wow,cant,heater,fine,perfectly,gets,simple
8,box,yard,patio,pleased,enough,size,splitting,around,done,looks
9,pump,idea,solid,deal,12,two,need,missing,make,keep


In [84]:
lda_display = pyLDAvis.gensim.prepare(lda_train4, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
